# Load Videos and estimate distance

In [1]:
import cv2 
import numpy as np 
import time
import matplotlib.pyplot as plt
import torch 
import importlib
import math
from pycocotools.coco import COCO
import skimage.io as io
import pylab
from sklearn.linear_model import LinearRegression
import os
import glob
import pandas as pd
from itertools import compress

dpt = importlib.import_module("Depth-Anything-V2.depth_anything_v2.dpt")
DepthAnythingV2 = dpt.DepthAnythingV2

util_transform = importlib.import_module("Depth-Anything-V2.depth_anything_v2.util.transform")

torch.cuda.is_available()

c:\Users\plima\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
c:\Users\plima\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


ModuleNotFoundError: No module named 'Depth-Anything-V2'

In [ ]:
# Parametros
real_height = 400   # mm
image_height = 1080 # pixels
sensor_height = 5.4 # mm  
focal_length = 32   # mm # Zoom
image_width = 1920 # pixels
# focal_length = 8   # mm # PTZ
# Ta trocado PTZ e panoramica


def calculate_distances(detected_circles, mode='reference'):

    if mode == 'pixel_count':
        distance_meters = []
        # https://photo.stackexchange.com/questions/12434/how-do-i-calculate-the-distance-of-an-object-in-a-photo
        dem = focal_length * real_height * image_height 
        for i in range(len(detected_circles)):
            radius = detected_circles[i][2]
            num = sensor_height * 2*radius 
            distance_meters.append(dem/num/1000)
    else: # Reference Caternary method
        # Reference distances in meters:
        left_distances = [ 97.03452097, 132.67746731, 182.81089   , 238.42127226,
                        296.38054131, 355.46744334, 415.11766024, 475.0301169 ,
                        535.02450033, 594.9826379 , 654.82177347, 714.48134854,
                        773.91599882, 833.09159591]
        right_distances = [ 90.47079257, 112.05005222, 156.71269605, 210.19780552,
                        267.20685889, 325.82715964, 385.24483613, 445.05429497,
                        505.02550385, 565.01390556, 624.92128397, 684.67714093,
                        744.22915224, 803.53794552]
        middle_of_image = image_width/2
        sorted_circles = sorted(detected_circles[0], key=lambda x: x[0])
        mask = np.array([circle[0]< middle_of_image for circle in sorted_circles])
        left_circles = list(compress(sorted_circles,mask))
        right_circles = list(compress(sorted_circles,~mask))
        distance_meters_left = np.zeros(len(left_circles))
        distance_meters_right = np.zeros(len(right_circles))
        
        for i in range(len(left_circles)):
            distance_meters_left[i] = left_distances[i]
        for i in range(len(right_circles)):
            distance_meters_right[len(right_circles)-i-1] = right_distances[i]
        distance_meters = np.concatenate((distance_meters_left, distance_meters_right), axis=0)

    return distance_meters

def check_depth_in_a_circle(depth, pt, folga = 0):
    x,y = pt[0],pt[1]
    r = pt[2]

    ix_min, ix_max = (np.max([np.floor(x-r),0]).astype(int), np.min([np.ceil(x+r),1918-1]).astype(int))
    iy_min, iy_max = (np.max([np.floor(y-r),0]).astype(int), np.min([np.ceil(y+r),1078-1]).astype(int))

    max = -999
    for ix in range(ix_min-folga, ix_max+folga):
        for iy in range(iy_min-folga, iy_max+folga):
            if( (ix-x)**2 + (iy-y)**2 <= (r+folga)**2):

                if depth[iy,ix] > max:
                    max = depth[iy,ix]

    return max


In [ ]:
videos = glob.glob("./DATASET/results/10-10-2024/*.avi")
CSVs =  glob.glob("./DATASET/results/10-10-2024/*.csv")
up_sampling = 10

[(CSVs[i], glob.glob(CSVs[i].split('track'+'*.avi')[0]) ) for i in range(len(CSVs))][:3]

[('./DATASET/results/10-10-2024\\10-10-2024_06-56-26_track_246_cam_0.avi_coordinates_tracking_242_cam_3.csv',
  ['./DATASET/results/10-10-2024\\10-10-2024_06-56-26_track_246_cam_0.avi_coordinates_tracking_242_cam_3.csv']),
 ('./DATASET/results/10-10-2024\\10-10-2024_07-14-32_track_248_cam_0.avi_coordinates_tracking_247_cam_4.csv',
  ['./DATASET/results/10-10-2024\\10-10-2024_07-14-32_track_248_cam_0.avi_coordinates_tracking_247_cam_4.csv']),
 ('./DATASET/results/10-10-2024\\10-10-2024_07-50-59_track_251_cam_0.avi_coordinates_tracking_249_cam_5.csv',
  ['./DATASET/results/10-10-2024\\10-10-2024_07-50-59_track_251_cam_0.avi_coordinates_tracking_249_cam_5.csv'])]

In [ ]:
dataDir='./DATASET/balls_distance_test'
# dataType='val2017'
annFile='{}/result.json'.format(dataDir)

image_id = 1

def annotatio_to_circles(coco, image_id,SAVE_IMG = False):
    # Take the anotted image and return the circles in it

    up_sampling = 10 # Up sample to better detect circles
    img_shape = [coco.imgs[0]['height'], coco.imgs[0]['width']]

    img_segmation = np.zeros([i*up_sampling for i in img_shape],dtype=np.uint8)
    seg_xy = []
    for annotation in coco.imgToAnns[image_id]:
        # annotation = coco.anns[key]
        # if annotation['image_id'] == image_id:
        seg_xy_up = np.array([ [annotation['segmentation'][0][i]*up_sampling, 
                                annotation['segmentation'][0][i+1]*up_sampling ] 
                                for i in range(0,len(annotation['segmentation'][0]),2)] 
                        ,dtype=np.int32)

        cv2.fillPoly(img_segmation, [seg_xy_up], 128)

        seg_xy.append( np.array([ [annotation['segmentation'][0][i], 
                            annotation['segmentation'][0][i+1] ] 
                                for i in range(0,len(annotation['segmentation'][0]),2)] 
                        ,dtype=np.int32)
                    )
            # cv2.polylines(image_segmentatted, [seg_xy], 1, (255,255,255), 1)

    detected_circles = cv2.HoughCircles(img_segmation,  
                   cv2.HOUGH_GRADIENT, 2, 20*up_sampling, 
                   param1 = 50,     param2 = 10, # param1 = 50, param2= 10
                   minRadius = 1*up_sampling,   maxRadius =10*up_sampling) 
    if SAVE_IMG:
        if detected_circles is not None: 
            detected_circles_sort = sorted(detected_circles[0,:], key=lambda x: x[0])
            distances = calculate_distances(detected_circles/up_sampling, mode='reference')
            for i,pt in enumerate(detected_circles_sort): 
                a, b, r = pt[0], pt[1], pt[2] 
                a, b, r = np.uint32((a,b,r))

                # Write distances
                text = str( round(distances[i]) ) + ' m;'
                # Draw the circumference of the circle. 
                cv2.circle(img_segmation,(a, b),r, 255, 1) 
                cv2.putText(img_segmation,text,(a,b),fontFace=1,fontScale=1,color=255)
        if not 'segmentation' in os.listdir('./results/'):
            os.mkdir('./results/segmentation')
        cv2.imwrite("./results/segmentation/"+coco.imgs[image_id]['file_name'].split('/')[-1].split('-',1)[1],img_segmation)


    return detected_circles, seg_xy


# detected_circles,seg_xy = annotatio_to_circles(coco, image_id)

In [ ]:
dataDir='./DATASET/balls_segmentation_out'
annFile='{}/result.json'.format(dataDir)
# initialize COCO api for instance annotations
coco=COCO(annFile)

img_shape = [coco.imgs[0]['width'], coco.imgs[0]['height']]
up_sampling = 10 # Up sample to better detect circles

balls_distance = {}
for image_id in coco.getImgIds():
    image_name = coco.imgs[image_id]['file_name'].split('/')[-1].split('-',1)[1]
    balls_distance[image_name] = {}
    
    print('processing image_id:',image_id,' from ', max(coco.getImgIds()) )
    detected_circles,seg_xy = annotatio_to_circles(coco, image_id,True)

    balls_distance[image_name]['detected_circles'] = detected_circles 
    balls_distance[image_name]['segmentation_coordinates'] = seg_xy


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
processing image_id: 0  from  6
processing image_id: 1  from  6
processing image_id: 2  from  6
processing image_id: 3  from  6
processing image_id: 4  from  6
processing image_id: 5  from  6
processing image_id: 6  from  6


In [ ]:
def create_depth(raw_image):
    # Create depth image
    # https://github.com/DepthAnything/Depth-Anything-V2?tab=readme-ov-file
    torch.cuda.empty_cache()    
    if(torch.cuda.memory_allocated()/1024/1024/1024 > 2):
        print("Memory allocated", torch.cuda.memory_allocated()/1024/1024/1024,"Gb")

    DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    if not torch.cuda.is_available():
        print("warning device:", DEVICE)
    
    raw_image_h,raw_image_w,_ = raw_image.shape
    model_configs = {
        'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
        'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
        'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
        'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
    }
        
    encoder = 'vitb' # or 'vits', 'vitl', 'vitg'

    depth_anything = DepthAnythingV2(**model_configs[encoder])
    depth_anything.load_state_dict(torch.load(f'Depth-Anything-V2/depth_anything_v2_{encoder}.pth', map_location='cpu'))
    depth_anything = depth_anything.to(DEVICE).eval()

    # Use model
    depth = depth_anything.infer_image(raw_image, np.floor(raw_image_h/14)*14)

    normalize = False
    # Transform in 
    if normalize:
        depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255.0
        depth = depth.astype(np.uint8)

    return depth

In [ ]:
def create_depth_to_meters(detected_circles, depth, SAVE_FIGS = False, raw_image = np.array([0]),folga=0):
    
    # Check depth in places that we now the
    vector_compare_radius_depth = []
    
    image_with_detection = raw_image.copy()

    if detected_circles is not None: 
        detected_circles_sort = sorted(detected_circles[0,:], key=lambda x: x[0])
        distances = calculate_distances(detected_circles/up_sampling, mode='reference')

        for i, pt in enumerate(detected_circles_sort): 
            a, b, r = pt[0]/up_sampling, pt[1]/up_sampling, pt[2]/up_sampling 
            a, b, r = np.uint32((a,b,r))

            depth_of_circle  = check_depth_in_a_circle(depth, (pt[0]/up_sampling, pt[1]/up_sampling, pt[2]/up_sampling),folga=20)

            # Write distances
            text = str( round(distances[i]) ) + ' m; ' \
                    + str(round(depth_of_circle,3)) + "depth"
            
            if SAVE_FIGS:
                # Draw the circumference of the circle. 
                cv2.circle(image_with_detection,(a, b),r+folga,(255, 0, 0), 1) 
                cv2.putText(image_with_detection,text,(a,b),fontFace=1,fontScale=1,color=255)
            
            # cv2.circle(processed_img,(a, b),r,(255, 0, 0), 1) 
            # cv2.putText(processed_img,text,(a,b),fontFace=1,fontScale=1,color=255)

            vector_compare_radius_depth.append([a, b, distances[i], depth_of_circle])


    ## Calculate the relationship between depth and distance
    distantece_y = np.array([ i[2] for i in vector_compare_radius_depth])
    depth_x = np.array([ [i[3]] for i in vector_compare_radius_depth])
    inv_depth_x = 1/depth_x
    


    distanceLR = LinearRegression().fit(inv_depth_x,distantece_y)
    new_vector_compare_radius_depth = []
    # new_depth_x = []
    # new_distantece_y = []
    new_vector_check = vector_compare_radius_depth
    if False: # Use or not use outliers
        while len(new_vector_check) != len(new_vector_compare_radius_depth) or \
            new_vector_check != new_vector_compare_radius_depth:

            new_vector_compare_radius_depth = []
            for eta in vector_compare_radius_depth:
                loss = (eta[3] * distanceLR.coef_[0] + distanceLR.intercept_ - eta[2])**2
                if loss < 1000:
                    new_vector_compare_radius_depth.append(eta)

            new_distantece_y = np.array([ i[2] for i in new_vector_compare_radius_depth])
            new_depth_x = np.array([ [i[3]] for i in new_vector_compare_radius_depth])

            distanceLR = LinearRegression().fit(new_depth_x,new_distantece_y)
            new_vector_check = new_vector_compare_radius_depth

    return [distanceLR.coef_ , distanceLR.intercept_], image_with_detection, (depth_x, distantece_y, new_vector_compare_radius_depth)


In [ ]:
def bird_distance(depth, coefs, track):
    
    if type(track['coordinates_x1']) == type('string'):
        ix_min = int(float(track['coordinates_x1'].replace(',','.')))
        ix_max = int(np.ceil(float(track['coordinates_x2'].replace(',','.'))))
        iy_min = int(float(track['coordinates_y1'].replace(',','.')))
        iy_max = int(np.ceil(float(track['coordinates_y2'].replace(',','.'))))
    else:
        ix_min = int(track['coordinates_x1'])
        ix_max = int(np.ceil(track['coordinates_x2']))
        iy_min = int(track['coordinates_y1'])
        iy_max = int(np.ceil(track['coordinates_y2']))

    max_depth = 0
    for ix in range(ix_min, ix_max):
        for iy in range(iy_min, iy_max):
            if depth[iy,ix] > max_depth:
                max_depth = depth[iy,ix]

    if max_depth > 0:    
        distance = (1/max_depth)*coefs[0] + coefs[1]
        return distance[0], max_depth
    return -1, max_depth


In [ ]:
def find_seg_img(filename,seg_file_names):
    for i, seg_name in enumerate(seg_file_names[:-1]):
        if filename > seg_name and filename < seg_file_names[i+1]:
            return seg_name
    return seg_file_names[-1]

seg_file_names = [coco.imgs[id]['file_name'].split('/')[-1].split('-',1)[1] for id in coco.getImgIds()]



In [ ]:
def draw_circles(frame, detected_circles, folga = 0):
    # print(len(detected_circles))
    # Draw circles that are detected. 
    if detected_circles is not None: 
        detected_circles_sort = sorted(detected_circles[0,:], key=lambda x: x[0])
        distances = calculate_distances(detected_circles/up_sampling, mode='reference')
        for i,pt in enumerate(detected_circles_sort): 
            a, b, r = pt[0]/up_sampling, pt[1]/up_sampling, pt[2]/up_sampling 
            a, b, r = np.uint32((a,b,r))

            depth_of_circle  = check_depth_in_a_circle(depth, (pt[0]/up_sampling, pt[1]/up_sampling, pt[2]/up_sampling))

            # Write distances
            text = str( round(distances[i] ) ) + '; ' +str(round(depth_of_circle))
            
            # Draw the circumference of the circle. 
            cv2.circle(frame,(a, b),r,(255, 0, 0), 1)
            if folga:
                cv2.circle(frame,(a, b),r+folga,(0, 255, 0),2) 
            cv2.putText(frame,text,(a,b),fontFace=1,fontScale=1,color=255)

    return frame

In [ ]:
def check_overlap(detected_circles, track, folga=0):
    # Return 1 if overlap

    if type(track['coordinates_x1']) == type('string'):
        ix_min = int(float(track['coordinates_x1'].replace(',','.')))
        ix_max = int(np.ceil(float(track['coordinates_x2'].replace(',','.'))))
        iy_min = int(float(track['coordinates_y1'].replace(',','.')))
        iy_max = int(np.ceil(float(track['coordinates_y2'].replace(',','.'))))
    else:
        ix_min = int(track['coordinates_x1'])
        ix_max = int(np.ceil(track['coordinates_x2']))
        iy_min = int(track['coordinates_y1'])
        iy_max = int(np.ceil(track['coordinates_y2']))

    corners = [
        [ix_min, iy_min],
        [ix_min, iy_max],
        [ix_max, iy_min],
        [ix_max, iy_max]
    ]


    for pt in detected_circles[0, :]: 
        a, b, r = pt[0]/up_sampling, pt[1]/up_sampling, pt[2]/up_sampling 
        for c in corners:
            if (a-c[0])**2 + (b-c[1])**2 <= (r+folga)**2:
                return 1
    
    return 0


In [ ]:
def create_outputs(filename, frame, depth, image_with_detection, bird_d, track, coefs, detected_circles, regression_points, seg_xy, folga, path = "./results/"):


    image_segmentatted = frame.copy()
    # image_with_detection = frame.copy()
    image_with_selected_detections = depth.copy()
    image_with_selected_detections = image_with_selected_detections/image_with_selected_detections.max()*255
    depth_w_bird = image_with_selected_detections.copy()

    if type(track['coordinates_x1']) == type('string'):
        ix_min = int(float(track['coordinates_x1'].replace(',','.')))
        ix_max = int(np.ceil(float(track['coordinates_x2'].replace(',','.'))))
        iy_min = int(float(track['coordinates_y1'].replace(',','.')))
        iy_max = int(np.ceil(float(track['coordinates_y2'].replace(',','.'))))
    else:
        ix_min = int(track['coordinates_x1'])
        ix_max = int(np.ceil(track['coordinates_x2']))
        iy_min = int(track['coordinates_y1'])
        iy_max = int(np.ceil(track['coordinates_y2']))

    image_with_detection = draw_circles(image_with_detection, detected_circles, folga)
    image_with_selected_detections = draw_circles(image_with_selected_detections, detected_circles, folga)
    cv2.rectangle(depth_w_bird,(ix_min,iy_min),(ix_max,iy_max),(255,255,255),2 )
    cv2.putText(depth_w_bird,str(round(bird_d,1))+" m",(ix_max,iy_max),fontFace=1,fontScale=1,color=255)

    cv2.rectangle(depth_w_bird,(ix_min,iy_min),(ix_max,iy_max),(255,255,255),2 )
    cv2.putText(depth_w_bird,str(round(bird_d,1))+" m",(ix_max,iy_max),fontFace=1,fontScale=1,color=255)
    (depth_x, distantece_y, new_vector_compare_radius_depth) = regression_points
    new_distantece_y = np.array([ i[2] for i in new_vector_compare_radius_depth])
    new_depth_x = np.array([ [i[3]] for i in new_vector_compare_radius_depth])

    for x,y,_,_ in new_vector_compare_radius_depth:
        cv2.circle(image_with_selected_detections, (x,y), 1,(0, 0, 255), 2) 


    plt.ioff()
    # print([seg_xy])
    cv2.fillPoly(image_segmentatted, seg_xy, 255)

    plt.figure(figsize=(12, 8))
    plt.subplot(3, 2, 1)
    plt.imshow(frame)
    plt.title("frame")
    plt.axis("off")

    plt.subplot(3, 2, 3)
    plt.imshow(image_segmentatted)
    plt.title("frame with ball Segmentation")
    plt.axis("off")

    plt.subplot(3, 2, 2)
    plt.imshow(depth_w_bird)
    plt.title("Depth with bird (d = " + str(round(bird_d,2)) + "m )")
    plt.axis("off")

    plt.subplot(3, 2, 4)
    plt.imshow(image_with_detection)
    plt.title("Circulos detectado")
    plt.axis("off")

    plt.subplot(3, 2, 6)
    plt.imshow(image_with_selected_detections)
    plt.title("Circulos utilizados (azul)")
    plt.axis("off")

    plt.subplot(3, 2, 5)
    plt.scatter(1/depth_x,distantece_y,c='b',alpha=.5)
    # plt.scatter(new_depth_x,new_distantece_y,c='k')
    plt.plot(1/depth_x, 1/depth_x * coefs[0][0] + coefs[1], 'r*')
    plt.xlabel("Inverso da Profundidade relativa (a="+str(np.round(coefs[0][0],2))+", b="+str(np.round(coefs[1],1))+")" )
    plt.ylabel("Distancia [m]")
    plt.title("Calculo da distancia")
    # plt.xlim((0, np.max(depth_x)+1))
    # plt.ylim(( 0 , np.max(distantece_y)+20))
    # plt.legend(["Todos valores","Valores selecionados","Regressão"])
    # plt.tight_layout()
    # plt.show()
    plt.savefig(path + filename +'.png',dpi=600)
    plt.close()


In [ ]:
def mask_cables(depth, th=.5):
    # Mask the cables
    mask = np.zeros(depth.shape, dtype=np.uint8)
    mask[depth > th] = 255
    return mask

def check_bird_in_mask(track, mask_cable, folga=10):
    # Check if the bird is in the mask
    ix_min = int(track['coordinates_x1'])
    ix_min_folga = int(track['coordinates_x1']) - folga
    ix_max = int(np.ceil(track['coordinates_x2']))
    ix_max_folga = int(np.ceil(track['coordinates_x2'])) + folga
    iy_min = int(track['coordinates_y1'])
    iy_min_folga = int(track['coordinates_y1']) - folga
    iy_max = int(np.ceil(track['coordinates_y2']))
    iy_max_folga = int(np.ceil(track['coordinates_y2'])) + folga
    # Check if the coordinates are within the image bounds
    if ix_min_folga < 0:
        ix_min_folga = 0
    if ix_max_folga > mask_cable.shape[1]:
        ix_max_folga = mask_cable.shape[1] 
    if iy_min_folga < 0:      
        iy_min_folga = 0  
    if iy_max_folga > mask_cable.shape[0]:    
        iy_max_folga = mask_cable.shape[0]    
    
    for ix in range(ix_min_folga, ix_max_folga):
        for iy in range(iy_min_folga, iy_max_folga):
            if ix >= ix_min and ix <= ix_max and iy >= iy_min and iy <= iy_max:
                continue
            if mask_cable[iy,ix] == 255:
                # print(iy,ix, "mask:", mask_cable[iy,ix]) 
                # print("depth:", depth[iy,ix], "_depth:", depth[ix,iy]) 
                # print("Bird in mask", track['coordinates_x1'],track['coordinates_x2'],track['coordinates_y1'],track['coordinates_y2'])
                return 1
    return 0

# check_bird_in_mask(track, mask_cable)

In [ ]:
tamanho_total = 0
for i,CSV in enumerate(CSVs):
    dfTracking = pd.read_csv(CSV)
    tamanho_total+= len(dfTracking)

print("Total de frames:", tamanho_total,"tempo estimado (8 segundos por frame):", round(tamanho_total*8/60/60,2),"horas")

Total de frames: 1767 tempo estimado (8 segundos por frame): 3.93 horas


In [ ]:
# Evaluate vídeos
folga = 20

ret = True
frame_number = 1
for i,CSV in enumerate(CSVs):
    if i!=1:
        continue
    dfTracking = pd.read_csv(CSV)

    video = glob.glob(CSV.split('track')[0]+'*.avi')
    if len(video) > 1:
        print(f"Warning on CSV {CSV} and video: {video}")
    video = video[0]
    
    print("video num:",i+1,"of",len(videos),". video file =",video.split('\\')[-1]," CSV file:",CSV.split('\\')[-1])
    
    dfDistance = pd.DataFrame(columns=dfTracking.columns) 
    dfDistance['video_file'] = []       # video file name
    dfDistance['bird_distance'] = []    # bird distance in meters
    dfDistance['bird_Depth'] = []       # bird relative depth 
    dfDistance['coefs'] = []            # coefficients for linear regression
    dfDistance['ref_depth_values'] = []     # ordered depth values for reference
    dfDistance['ref_distance_values'] = []  # ordered distance values for reference
    dfDistance['ref_choosen_points'] = []   # choosen points of reference
    dfDistance['ref_circles'] = []           # Name of the reference image for circle sizes
    dfDistance['ignore'] = []           # Name of the reference image for circle sizes
    circles_ref = find_seg_img(video,seg_file_names)
    detected_circles = balls_distance[circles_ref]['detected_circles']
    
    cap = cv2.VideoCapture(video)

    for j, track in dfTracking.iterrows():

        # start = time.time()
        cap.set(cv2.CAP_PROP_POS_FRAMES, track['current_frame']-1)
        ret, frame = cap.read()

        if not ret:
            print("frame ",str(track['current_frame']), " não lido")
            continue
        depth = create_depth(frame)

        mask_cable = mask_cables(depth)

        coefs, image_with_detection, regression_points = create_depth_to_meters(detected_circles, depth,SAVE_FIGS = True, raw_image = frame)
        bird_d, bird_depth = bird_distance(depth, coefs, track)

        (depth_x, distantece_y, new_vector_compare_radius_depth) = regression_points

        toDataframe = list(track.values)
        toDataframe.append(video.split('\\')[-1])
        toDataframe.append(bird_d)
        toDataframe.append(bird_depth)            #bird_Depth
        toDataframe.append(coefs)  #coefs
        toDataframe.append(depth_x)            #ref_depth_values
        toDataframe.append(distantece_y)            #ref_distance_values
        toDataframe.append(new_vector_compare_radius_depth)            #ref_choosen_points
        toDataframe.append(circles_ref)            # ref_circles
        toDataframe.append(check_overlap(detected_circles,track,folga) # balls
                           or check_bird_in_mask(track, mask_cable) # Mask
                           ) # ignore
        dfDistance.loc[len(dfDistance)] = toDataframe

        filename = video.split('\\')[-1]+"_frame_"+str(track['current_frame'])
        if True:
            create_outputs(filename, frame, depth, image_with_detection, bird_d, track, coefs, detected_circles, 
                           regression_points, balls_distance[circles_ref]['segmentation_coordinates'], folga, 
                           path = "./results/")
        # break
        print(f'frame {j+1} of {len(dfTracking)}------- img filename: ',filename,end='\r')

    cap.release()
    # Save a CSV per video
    dfDistance.to_csv("./results/"+ video.split('\\')[-1] + ".csv")


cv2.destroyAllWindows()

video num: 3 of 24 . video file = 10-10-2024_07-50-59_track_251_cam_0.avi_track_249_cam_5.avi  CSV file: 10-10-2024_07-50-59_track_251_cam_0.avi_coordinates_tracking_249_cam_5.csv


Rascunhos

In [ ]:
def show_mask():
    folga = 5
    ix_min = int(track['coordinates_x1'])
    ix_min_folga = int(track['coordinates_x1']) - folga
    ix_max = int(np.ceil(track['coordinates_x2']))
    ix_max_folga = int(np.ceil(track['coordinates_x2'])) + folga
    iy_min = int(track['coordinates_y1'])
    iy_min_folga = int(track['coordinates_y1']) - folga
    iy_max = int(np.ceil(track['coordinates_y2']))
    iy_max_folga = int(np.ceil(track['coordinates_y2'])) + folga

    cv2.rectangle(depth,(ix_min,iy_min),(ix_max,iy_max),(255,255,255),2 )
    cv2.rectangle(depth,(ix_min_folga,iy_min_folga),(ix_max_folga,iy_max_folga),(255,255,255),2 )

    cv2.imshow('depth',mask_cables(depth,th=.5))
    cv2.waitKey(0)
    cv2.destroyAllWindows()